In [77]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow.keras import regularizers

In [78]:
# Load and preprocess
file_path = '../../csv/features_data-Sheet1.csv'

In [79]:

df = pd.read_csv(file_path)
#   X = df[['length', 'width', 'diameter', 'area', 'perimeter', 'rectangularity', 'circularity',
#           'ellipticity', 'aspect ratio', 'form factor', 'narrow factor', 'perimeter ratio of diameter',
#           'perimeter Ratio of Physiological Length and Physiological Width', 'Texture']]
#   y = df['Species']

X = df[['circularity',
            'ellipticity', 'aspect ratio', 'form factor', 'narrow factor', 'perimeter ratio of diameter',
            'perimeter Ratio of Physiological Length and Physiological Width']]
y = df['Species']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [80]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=64, input_dim=X_train.shape[1], activation = 'relu'))
model.add(tf.keras.layers.Dense(units=32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(tf.keras.layers.Dense(units=1, activation='linear'))

In [86]:
# Compile the model
from tensorflow import keras
model.compile(optimizer='adam', loss='mean_squared_error', metrics = ['accuracy'])

In [95]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

es = EarlyStopping(monitor = 'test_accuracy', min_delta=0.01, patience = 3, verbose = 1)

mc = ModelCheckpoint(filepath = '../../models/plant_prediction_model_ann.h5', 
                     monitor = 'accuracy', 
                     min_delta=0.01, 
                     patience = 2, 
                     verbose = 1, 
                     save_best_only = True)

cb = [es, mc]

In [96]:
his = model.fit(X_train, y_train, steps_per_epoch = 20,  epochs = 20,  verbose = 1,  callbacks = cb, validation_steps = 16)

Epoch 1/20
20/20 [==============================] - ETA: 0s - loss: 0.8639 - accuracy: 0.2517WARNING:tensorflow:Early stopping conditioned on metric `test_accuracy` which is not available. Available metrics are: loss,accuracy

Epoch 00001: accuracy improved from -inf to 0.25169, saving model to ../../models/plant_prediction_model_ann.h5
20/20 [==============================] - 0s 8ms/step - loss: 0.8639 - accuracy: 0.2517
Epoch 2/20
14/20 [====================>.........] - ETA: 0s - loss: 0.8501 - accuracy: 0.2664WARNING:tensorflow:Early stopping conditioned on metric `test_accuracy` which is not available. Available metrics are: loss,accuracy

Epoch 00002: accuracy improved from 0.25169 to 0.25846, saving model to ../../models/plant_prediction_model_ann.h5
20/20 [==============================] - 0s 5ms/step - loss: 0.8189 - accuracy: 0.2585
Epoch 3/20
13/20 [==================>...........] - ETA: 0s - loss: 0.8534 - accuracy: 0.2516WARNING:tensorflow:Early stopping conditioned on met

In [85]:
# Evaluate the model on the testing set
loss, accuracy = model.evaluate(X_test, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

3/3 [==============================] - 0s 4ms/step - loss: -7.2868 - accuracy: 0.1687
Test Loss: -7.28681755065918
Test Accuracy: 0.16867469251155853


In [65]:
# Make predictions with the trained model
predictions = model.predict(X_test)
# Convert probabilities to binary predictions
binary_predictions = np.round(predictions)

In [67]:
# Calculate precision and recall
precision = precision_score(y_test, binary_predictions, average='weighted')
recall = recall_score(y_test, binary_predictions, average='weighted')

d:\anaconda3\envs\tf2.4\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [68]:
print('Precision:', precision)
print('Recall:', recall)

Precision: 0.05240238060676441
Recall: 0.2289156626506024
